In [ ]:
#Reloads the lab.py and crystals.py modules to update any changes (after saving)
#If a new method or object is created, autoreload doesn't work and the 
#kernel needs to be closed and halted after saving and making a 'checkpoint'
#in this notebook

%load_ext autoreload
%autoreload 2

In [ ]:
import ipas
import matplotlib.pyplot as plt
import numpy as np
from sqlite3 import Connection as SQLite3Connection
from sqlalchemy import create_engine, event, select
from sqlalchemy.engine import Engine
from sqlalchemy.orm import sessionmaker
import base
from ipas import lab_ice_agg_SQL as lab
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
from dask import delayed
import cloudpickle as pickle
from multiprocessing import Pool
from functools import partial
import time
import seaborn as sns
from matplotlib.colors import LogNorm
import scipy.stats as stats 
import matplotlib as mpl

In [ ]:
#14 days batch 
#5 days kratos
#cluster = SLURMCluster(
#    queue='kratos',
#    walltime='04-23:00:00',
#    cores=1,
##    memory='4096MiB', #1 GiB = 1,024 MiB
#    processes=1)

cluster = SLURMCluster(
    queue='batch',
    walltime='02-23:00:00',
    cores=1,
    memory='7168MiB', #1 GiB = 1,024 MiB
    processes=1)


cluster.scale(1)
#2 nodes 128 GiB 

In [ ]:
client = Client(cluster)

In [ ]:
client

In [ ]:
phioarr=np.logspace(-2, 2, num=20, dtype=None)#just columns (0,2); plates (-2,0)
#phioarr = phioarr[1:]
lenphio = len(phioarr)

reqarr = [6,7,8,9]
numaspectratios=len(phioarr)
ch_dist='gamma'         #anything other than gamma uses the characteristic from the best distribution pdf (lowest SSE)
nclusters = 300        #changes how many aggregates per aspect ratio to consider
ncrystals = 50       #number of monomers per aggregate 1
minor = 'depth'        #'minorxy' from fit ellipse or 'depth' to mimic IPAS in IDL
rand_orient = True    #randomly orient the seed crystal and new crystal: uses first random orientation
save_plots = False     #saves all histograms to path with # of aggs and minor/depth folder
file_ext = 'eps'

In [ ]:
def main():
    '''
    engine = create_engine('sqlite:///IPAS_1_lastmono.sqlite')
    event.listen(engine, 'connect', _set_sqlite_pragma)
    
    base.Base.metadata.create_all(engine, checkfirst=True)
    Session = sessionmaker(bind=engine)
    session = Session()

    for phio in phioarr:
        for r in reqarr:
            b1 = lab.collect_clusters(phio, r, session, nclusters=nclusters,
                                ncrystals=ncrystals, rand_orient=rand_orient)
    
    '''
    notebook=1
    for r in reqarr:
        #b3 = []
        print('r = ',r)
        count = 1
        pool = Pool(processes=20) #pools are reusable
        parallel_clus=partial(lab.collect_clusters, notebook=notebook, r=r, nclusters=nclusters,\
                                ncrystals=ncrystals, \
                                rand_orient=rand_orient)

        start = time.time()
        output = pool.imap_unordered(parallel_clus, phioarr)
        for done in output:
            print("after %3.1fsec: count:%s"%(time.time()-start, count))
            count +=1

#         for res in output:
#             print("(after %3.1fsec)  mono phi:%.3f  count:%s"%(time.time()-start, res[0].mono_phi, count))
#             count += 1
        print('closing')
        pool.close()
        print('joining')
        pool.join()
        print('new r')
    '''
    
    output = []
    for r in reqarr:
        for phio in phioarr:
    
            print('eq. vol rad', r, phio)

            output.append(delayed(lab.collect_clusters)(phio, r, nclusters=nclusters,
                                                        ncrystals=ncrystals, 
                                                        rand_orient=rand_orient))
            
    print(output)
    #print('computing...')
    b1 = client.compute(output, scheduler='distributed') 

    print('-------------gathering-----------')
    b1 = client.gather(b1)
    print('done gathering')
    '''



In [ ]:
if __name__ == '__main__':
    
    %time main() 
     
#     filename = 'instance_3radii_iceagg_allrand_50xtalstot_20phi_rall_300agg_1_lastmono'
#     filehandler = open(filename, 'wb')
#     %time pickle.dump(b1, filehandler)
#     filehandler.close()
#     print('finished, creating engine 1!')
    
#     engine = create_engine('sqlite:///IPAS_1_lastmono.sqlite')
#     event.listen(engine, 'connect', _set_sqlite_pragma)
    
#     base.Base.metadata.create_all(engine, checkfirst=True)
#     Session = sessionmaker(bind=engine)
#     session = Session()
    
#     try:
#         for obj in b1:
#             session.add_all(obj)  # crystal id has been appended into cluster relationship
#             session.commit()

#     except:
#         print('in except')
#         raise

      
#     print('DONE!!!!!')


In [ ]:
filename = 'instance_3radii_iceagg_allrand_50xtalstot_20phi_rall_300agg_1_lastmono'
filehandler = open(filename, 'wb')
%time pickle.dump(b1, filehandler)
filehandler.close()
print('finished, creating engine 1!')

engine = create_engine('sqlite:///IPAS_1_lastmono.sqlite')
event.listen(engine, 'connect', _set_sqlite_pragma)

base.Base.metadata.create_all(engine, checkfirst=True)
Session = sessionmaker(bind=engine)
session = Session()

try:
    for obj in b1:
        session.add_all(obj)  # crystal id has been appended into cluster relationship
        session.commit()

except:
    print('in except')
    raise

In [ ]:
engine = create_engine('sqlite:///IPAS_1_lastmono.sqlite')
event.listen(engine, 'connect', _set_sqlite_pragma)

base.Base.metadata.create_all(engine, checkfirst=True)
Session = sessionmaker(bind=engine)
session = Session()

print('AGGREGATE PROPERTIES ==============')

query = session.query(ipas.IceCluster)#, crys.IceCrystal)#.filter(crys.IceCrystal.id == 10)
#query=session.query(clus.IceCluster).filter(crystals = 1).all()
for agg in query.all():
    print(agg.id, agg.ncrystals, agg.agg_phi, agg.cplx, agg.crystal[0].points,  agg.crystal[0].phi, agg.crystal[0].r)

print('MONOMER PROPERTIES ================')
query = session.query(ipas.IceCrystal)
for mono in query.all():
    print(mono.id, mono.phi, mono.r, mono.rand_orient, mono.aggs.ncrystals, mono.aggs.agg_phi)

# try:
#     for r in b1:
#         for obj in r:
#             session.add_all(obj)  # crystal id has been appended into cluster relationship
#             session.commit()

# except:
#     print('in except')
#     raise

# session.close()  
# print('DONE!!!!!')
 